In [222]:
import sbmlcore, pandas, numpy, pytest
import freesasa
import os.path
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Documentation on installing: https://freesasa.github.io/ </br>
Documentation on using: https://freesasa.github.io/python/intro.html </br>
Documentation on classes: https://freesasa.github.io/python/classes.html </br>
Selection syntax (N.B. uses Pymol syntax!): https://freesasa.github.io/doxygen/Selection.html

In [141]:
#Test for pncA structure
structure = freesasa.Structure("tests/3pl1.pdb")
result = freesasa.calc(structure)
area_classes = freesasa.classifyResults(result, structure)

print("Total : %.2f A2" % result.totalArea())
for key in area_classes:
    print(key, ": %.2f A2" % area_classes[key])
    
print("Total number of atoms: ", result.nAtoms())
print("Atom area for atom 5 in A^2", result.atomArea(5))
#print("Residue area for resid 5", result.residueAreas()) #seems to return objects, not floats...

res_areas = result.residueAreas()
print(res_areas["A"]["5"])

#To get SASA of different residues, uses Pymol syntax
# ('selection-name, selector', 'selection-name, selector')
# Notes on syntax are here: https://freesasa.github.io/doxygen/Selection.html
selections = freesasa.selectArea(('alanine, resn ala', 'r1_10, resi 1-10'), 
                                 structure, result)
for key in selections:
    print(key, ": %.2f A2" % selections[key])

first_residue = freesasa.selectArea(('alanine, resn ala', 'proline, resn pro'), structure, result)
first_residue = freesasa.selectArea(['r1, resi 1', 'r1_10, resi 1-10'], 
                                 structure, result)
print(first_residue)

second_third_fourth_residue = freesasa.selectArea(('resid2, chain A and resi 2', 'r3, resi 3', 'r4, resi 4'), 
                                 structure, result)

print(second_third_fourth_residue)
print(type(second_third_fourth_residue))

b = {'segid': ['A', 'A', 'A'], 'mutation': ['M1D','R2K', 'A3V']}
df = pandas.DataFrame.from_dict(b)
df

# [i for i in range(10)]
#test = freesasa.Classifier.classify('alanine, resn ala') #Doesn't take str as argument, but documentation says it does? https://freesasa.github.io/python/classes.html 

Total : 8544.47 A2
Polar : 3938.05 A2
Apolar : 4606.42 A2
Total number of atoms:  1372
Atom area for atom 5 in A^2 14.037357366842954
alanine : 707.16 A2
r1_10 : 159.29 A2
{'r1': 96.20442808708866, 'r1_10': 159.289093338855}
{'resid2': 57.3917691721817, 'r3': 0.0, 'r4': 0.1038304284352447}
<class 'dict'>


,segid,mutation
0,A,M1D
1,A,R2K
2,A,A3V


In [142]:
def split_mutation(row):
    m=row.mutation
    return(int(m[1:-1]))

df['resid'] = df.apply(split_mutation, axis=1)
df['id'] = df['segid'] + df['resid'].astype(str)
df.set_index('id', inplace=True)
df

,segid,mutation,resid
id,,,
A1,A,M1D,1
A2,A,R2K,2
A3,A,A3V,3


In [143]:
sele_text = ["%s%i, resi %i and chain %s" % (j,i,i,j) for i,j in zip(df.resid, df.segid)]
sele_text

['A1, resi 1 and chain A', 'A2, resi 2 and chain A', 'A3, resi 3 and chain A']

In [144]:
results = freesasa.selectArea(sele_text, structure, result)
print(results)
s = pandas.Series(results)
b = pandas.DataFrame(s, columns=['surface_area'])
b

{'A1': 96.20442808708866, 'A2': 57.3917691721817, 'A3': 0.0}


,surface_area
A1,96.204428
A2,57.391769
A3,0.000000


In [145]:
df.join(b, how='left')

,segid,mutation,resid,surface_area
id,,,,
A1,A,M1D,1,96.204428
A2,A,R2K,2,57.391769
A3,A,A3V,3,0.000000


In [146]:
#Test for RNAP structure
structure = freesasa.Structure("tests/5uh6.pdb")
result = freesasa.calc(structure)
area_classes = freesasa.classifyResults(result, structure)

print("Total : %.2f A2" % result.totalArea())
for key in area_classes:
    print(key, ": %.2f A2" % area_classes[key])

Total : 140821.76 A2
Polar : 68137.62 A2
Apolar : 72684.14 A2


In [147]:
freesasa

<module 'freesasa' from '/opt/homebrew/Caskroom/miniconda/base/envs/sbmlcore/lib/python3.9/site-packages/freesasa.cpython-39-darwin.so'>

In [244]:
#Test on RNAP structure with offsets
file = sbmlcore.FreeSASA('tests/5uh6.pdb', offsets = {'A': 0, 'B': 0, 'Z': -6}) #
b = {'segid': ['A', 'A', 'A', 'B', 'C', 'C'], 'mutation': ['I3D','S4K', 'Q5V', 'R6D', 'S450F', 'D435F']} #N.B. Mutation must include offset
df = pandas.DataFrame(b)
df

,segid,mutation
0,A,I3D
1,A,S4K
2,A,Q5V
3,B,R6D
4,C,S450F
5,C,D435F


In [245]:
new_df = file.add_feature(df) #, offsets = {'A': 0, 'B': 0, 'C': -6}
new_df

AssertionError: Need to specify a segid that exists in pdb!

In [173]:
#Test on pncA
a = sbmlcore.FreeSASA('tests/3pl1.pdb')

b = {'segid': ['A', 'A', 'A'], 'mutation': ['M1D','R2K', 'A3V']}
df = pandas.DataFrame.from_dict(b)
df

,segid,mutation
0,A,M1D
1,A,R2K
2,A,A3V


In [174]:
df = a.add_feature(df)
df

,segid,mutation,resid,surface_area
id,,,,
A1,A,M1D,1,96.204428
A2,A,R2K,2,57.391769
A3,A,A3V,3,0.000000
